In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


# Load Toronto datasets from Part 2

In [2]:
df = pd.read_csv('datasets/toronto_part2.csv')
df.head()

PostalCode      Borough                            Neighborhood   Latitude  \
0        M1B  Scarborough                          Rouge, Malvern  43.806686   
1        M1C  Scarborough  Highland Creek, Rouge Hill, Port Union  43.784535   
2        M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3        M1G  Scarborough                                  Woburn  43.770992   
4        M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476

# Toronto Geolocation

In [5]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="canada_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


# Create a map of Toronto

In [6]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Select only Toronto borough

In [7]:
df_toronto = df[df['Borough'].str.contains("Toronto")].reset_index().drop(['index','PostalCode'],axis=1)

In [8]:
df_toronto.head()

Borough                    Neighborhood   Latitude  Longitude
0     East Toronto                     The Beaches  43.676357 -79.293031
1     East Toronto    The Danforth West, Riverdale  43.679557 -79.352188
2     East Toronto  The Beaches West, India Bazaar  43.668999 -79.315572
3     East Toronto                 Studio District  43.659526 -79.340923
4  Central Toronto                   Lawrence Park  43.728020 -79.388790

In [9]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], 
                                           df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Foursquare API

In [33]:
CLIENT_ID = 'DELETE-BEFORE-COMMIT' # your Foursquare ID
CLIENT_SECRET = 'DELETE-BEFORE-COMMIT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DELETE-BEFORE-COMMIT
CLIENT_SECRET:DELETE-BEFORE-COMMIT


# Explore Neighborhood

In [11]:
df_toronto.loc[0, 'Neighborhood']

'The Beaches'

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
radius=500
LIMIT=100

toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'], latitudes=df_toronto['Latitude'], 
                                 longitudes=df_toronto['Longitude'])

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The Junction Sout

In [15]:
print(toronto_venues.shape)
toronto_venues.head()

(1706, 7)


Neighborhood  Neighborhood Latitude  \
0                   The Beaches              43.676357   
1                   The Beaches              43.676357   
2                   The Beaches              43.676357   
3                   The Beaches              43.676357   
4  The Danforth West, Riverdale              43.679557   

   Neighborhood Longitude                               Venue  Venue Latitude  \
0              -79.293031                   Glen Manor Ravine       43.676821   
1              -79.293031  The Big Carrot Natural Food Market       43.678879   
2              -79.293031                 Grover Pub and Grub       43.679181   
3              -79.293031                       Upper Beaches       43.680563   
4              -79.352188                            Pantheon       43.677621   

   Venue Longitude     Venue Category  
0       -79.293942              Trail  
1       -79.297734  Health Food Store  
2       -79.297215                Pub  
3       -79.292869       Neighborhood  
4       -79.351434   Greek Restaurant

In [16]:
toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Adelaide, King, Richmond                                              100   
Berczy Park                                                            55   
Brockton, Exhibition Place, Parkdale Village                           24   
Business Reply Mail Processing Centre 969 Eastern                      17   
CN Tower, Bathurst Quay, Island airport, Harbou...                     16   
Cabbagetown, St. James Town                                            42   
Central Bay Street                                                     84   
Chinatown, Grange Park, Kensington Market                              80   
Christie                                                               18   
Church and Wellesley                                                   81   
Commerce Court, Victoria Hotel                                        100   
Davisville                                                             35   
Davisville North                                                        7   
Deer Park, Forest Hill SE, Rathnelly, South Hil...                     14   
Design Exchange, Toronto Dominion Centre                              100   
Dovercourt Village, Dufferin                                           19   
First Canadian Place, Underground city                                100   
Forest Hill North, Forest Hill West                                     4   
Harbord, University of Toronto                                         36   
Harbourfront                                                           46   
Harbourfront East, Toronto Islands, Union Station                     100   
High Park, The Junction South                                          25   
Lawrence Park                                                           4   
Little Portugal, Trinity                                               51   
Moore Park, Summerhill East                                             4   
North Toronto West                                                     23   
Parkdale, Roncesvalles                                                 14   
Queen's Park                                                           39   
Rosedale                                                                4   
Roselawn                                                                3   
Runnymede, Swansea                                                     40   
Ryerson, Garden District                                              100   
St. James Town                                                        100   
Stn A PO Boxes 25 The Esplanade                                        95   
Studio District                                                        41   
The Annex, North Midtown, Yorkville                                    21   
The Beaches                                                             4   
The Beaches West, India Bazaar                                         18   
The Danforth West, Riverdale                                           42   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
Adelaide, King, Richmond                                               100   
Berczy Park                                                             55   
Brockton, Exhibition Place, Parkdale Village                            24   
Business Reply Mail Processing Centre 969 Eastern                       17   
CN Tower, Bathurst Quay, Island airport, Harbou...                      16   
Cabbagetown, St. James Town                                             42   
Central Bay Street                                                      84   
Chinatown, Grange Park, Kensington Market                               80   
Christie                                                                18   
Church and Wellesley                

In [17]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 229 uniques categories.


# Analyze each neighborhood

In [18]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

Yoga Studio  Afghan Restaurant  Airport  Airport Food Court  Airport Gate  \
0            0                  0        0                   0             0   
1            0                  0        0                   0             0   
2            0                  0        0                   0             0   
3            0                  0        0                   0             0   
4            0                  0        0                   0             0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0               0                0                 0                    0   
1               0                0                 0                    0   
2               0                0                 0                    0   
3               0                0                 0                    0   
4               0                0                 0                    0   

   Antique Shop  Aquarium  Art Gallery  Arts & Crafts Store  Asian Restaurant  \
0             0         0            0                    0                 0   
1             0         0            0                    0                 0   
2             0         0            0                    0                 0   
3             0         0            0                    0                 0   
4             0         0            0                    0                 0   

   Athletics & Sports  Auto Workshop  BBQ Joint  Baby Store  Bagel Shop  \
0                   0              0          0           0           0   
1                   0              0          0           0           0   
2                   0              0          0           0           0   
3                   0              0          0           0           0   
4                   0              0          0           0           0   

   Bakery  Bank  Bar  Baseball Stadium  Basketball Stadium  Beach  \
0       0     0    0                 0                   0      0   
1       0     0    0                 0                   0      0   
2       0     0    0                 0                   0      0   
3       0     0    0                 0                   0      0   
4       0     0    0                 0                   0      0   

   Bed & Breakfast  Beer Bar  Beer Store  Belgian Restaurant  Bistro  \
0                0         0           0                   0       0   
1                0         0           0                   0       0   
2                0         0           0                   0       0   
3                0         0           0                   0       0   
4                0         0           0                   0       0   

   Board Shop  Boat or Ferry  Bookstore  Boutique  Brazilian Restaurant  \
0           0              0          0         0                     0   
1           0              0          0         0                     0   
2           0              0          0         0                     0   
3           0              0          0         0                     0   
4           0              0          0         0                     0   

   Breakfast Spot  Brewery  Bubble Tea Shop  Building  Burger Joint  \
0               0        0                0         0             0   
1               0        0                0         0             0   
2               0        0                0         0             0   
3               0        0                0         0             0   
4               0        0                0         0             0   

   Burrito Place  Bus Line  Butcher  Café  Cajun / Creole Restaurant  \
0              0         0        0     0                          0   
1              0         0        0     0                          0   
2              0         0        0     0                          0   
3              0         0        0     0                          0   
4              0         0        0

In [19]:
toronto_onehot.shape

(1706, 229)

In [20]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

Neighborhood  Yoga Studio  \
0                            Adelaide, King, Richmond     0.000000   
1                                         Berczy Park     0.000000   
2        Brockton, Exhibition Place, Parkdale Village     0.041667   
3   Business Reply Mail Processing Centre 969 Eastern     0.000000   
4   CN Tower, Bathurst Quay, Island airport, Harbo...     0.000000   
5                         Cabbagetown, St. James Town     0.000000   
6                                  Central Bay Street     0.011905   
7           Chinatown, Grange Park, Kensington Market     0.000000   
8                                            Christie     0.000000   
9                                Church and Wellesley     0.012346   
10                     Commerce Court, Victoria Hotel     0.000000   
11                                         Davisville     0.000000   
12                                   Davisville North     0.000000   
13  Deer Park, Forest Hill SE, Rathnelly, South Hi...     0.000000   
14           Design Exchange, Toronto Dominion Centre     0.000000   
15                       Dovercourt Village, Dufferin     0.000000   
16             First Canadian Place, Underground city     0.000000   
17                Forest Hill North, Forest Hill West     0.000000   
18                     Harbord, University of Toronto     0.027778   
19                                       Harbourfront     0.021739   
20  Harbourfront East, Toronto Islands, Union Station     0.000000   
21                      High Park, The Junction South     0.000000   
22                                      Lawrence Park     0.000000   
23                           Little Portugal, Trinity     0.019608   
24                        Moore Park, Summerhill East     0.000000   
25                                 North Toronto West     0.043478   
26                             Parkdale, Roncesvalles     0.000000   
27                                       Queen's Park     0.025641   
28                                           Rosedale     0.000000   
29                                           Roselawn     0.000000   
30                                 Runnymede, Swansea     0.025000   
31                           Ryerson, Garden District     0.000000   
32                                     St. James Town     0.000000   
33                    Stn A PO Boxes 25 The Esplanade     0.000000   
34                                    Studio District     0.024390   
35                The Annex, North Midtown, Yorkville     0.000000   
36                                        The Beaches     0.000000   
37                     The Beaches West, India Bazaar     0.000000   
38                       The Danforth West, Riverdale     0.023810   

    Afghan Restaurant  Airport  Airport Food Court  Airport Gate  \
0            0.000000   0.0000              0.0000        0.0000   
1            0.000000   0.0000              0.0000        0.0000   
2            0.000000   0.0000              0.0000        0.0000   
3            0.000000   0.0000              0.0000        0.0000   
4            0.000000   0.0625              0.0625        0.0625   
5            0.000000   0.0000              0.0000        0.0000   
6            0.000000   0.0000              0.0000        0.0000   
7            0.000000   0.0000              0.0000        0.0000   
8            0.000000   0.0000              0.0000        0.0000   
9            0.012346   0.0000              0.0000        0.0000   
10           0.000000   0.0000              0.0000        0.0000   
11           0.000000   0.0000              0.0000        0.0000   
12           0.000000   0.0000              0.0000        0.0000   
13           0.000000   0.0000              0.0000        0.0000   
14           0.000000   0.0000              0.0000        0.0000   
15           0.000000   0.0000              0.0000        0.0000   
16           0.000000   0.0000              0.0000        0.0000   
17           0.0

In [21]:
toronto_grouped.shape

(39, 229)

In [22]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
              venue  freq
0       Coffee Shop  0.07
1               Bar  0.04
2        Steakhouse  0.04
3              Café  0.04
4  Asian Restaurant  0.03


----Berczy Park----
          venue  freq
0   Coffee Shop  0.07
1  Cocktail Bar  0.05
2          Café  0.04
3    Steakhouse  0.04
4      Beer Bar  0.04


----Brockton, Exhibition Place, Parkdale Village----
            venue  freq
0  Breakfast Spot  0.08
1            Café  0.08
2     Coffee Shop  0.08
3       Nightclub  0.08
4     Yoga Studio  0.04


----Business Reply Mail Processing Centre 969 Eastern----
                  venue  freq
0      Recording Studio  0.06
1         Garden Center  0.06
2  Fast Food Restaurant  0.06
3            Skate Park  0.06
4                  Park  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
                 venue  freq
0      Airport Service  0.19
1     Airport Terminal  0.12
2       Airpor

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [24]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                           Adelaide, King, Richmond           Coffee Shop   
1                                        Berczy Park           Coffee Shop   
2       Brockton, Exhibition Place, Parkdale Village           Coffee Shop   
3  Business Reply Mail Processing Centre 969 Eastern                  Park   
4  CN Tower, Bathurst Quay, Island airport, Harbo...       Airport Service   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0            Steakhouse                   Bar                  Café   
1          Cocktail Bar        Farmers Market    Seafood Restaurant   
2                  Café        Breakfast Spot             Nightclub   
3         Burrito Place               Butcher  Fast Food Restaurant   
4      Airport Terminal        Airport Lounge      Sculpture Garden   

  5th Most Common Venue 6th Most Common Venue  7th Most Common Venue  \
0        Cosmetics Shop        Breakfast Spot       Asian Restaurant   
1            Steakhouse                  Café            Cheese Shop   
2           Yoga Studio                Bakery  Performing Arts Venue   
3        Farmers Market         Auto Workshop       Recording Studio   
4   Rental Car Location       Harbor / Marina                    Bar   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0          Burger Joint            Restaurant        Thai Restaurant  
1              Beer Bar                Bakery         Breakfast Spot  
2             Pet Store     Convenience Store           Climbing Gym  
3                   Spa            Restaurant            Pizza Place  
4         Boat or Ferry           Coffee Shop           Airport Gate

# Cluster neighborhoods

In [25]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [26]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

Borough                    Neighborhood   Latitude  Longitude  \
0     East Toronto                     The Beaches  43.676357 -79.293031   
1     East Toronto    The Danforth West, Riverdale  43.679557 -79.352188   
2     East Toronto  The Beaches West, India Bazaar  43.668999 -79.315572   
3     East Toronto                 Studio District  43.659526 -79.340923   
4  Central Toronto                   Lawrence Park  43.728020 -79.388790   

   Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0               4                 Trail     Health Food Store   
1               0      Greek Restaurant           Coffee Shop   
2               0                  Park      Sushi Restaurant   
3               0                  Café           Coffee Shop   
4               2              Bus Line                  Lake   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0                   Pub         Women's Store          Dance Studio   
1    Italian Restaurant            Restaurant        Ice Cream Shop   
2            Board Shop               Brewery    Italian Restaurant   
3             Gastropub                Bakery               Brewery   
4           Swim School                  Park  Colombian Restaurant   

         6th Most Common Venue 7th Most Common Venue  \
0  Eastern European Restaurant   Dumpling Restaurant   
1       Furniture / Home Store           Yoga Studio   
2                 Liquor Store          Burger Joint   
3           Italian Restaurant   American Restaurant   
4      Comfort Food Restaurant     Electronics Store   

         8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0                   Donut Shop      Doner Restaurant                Dog Run  
1                          Pub                Bakery           Dessert Shop  
2                Burrito Place        Ice Cream Shop   Fast Food Restaurant  
3      Comfort Food Restaurant        Sandwich Place            Cheese Shop  
4  Eastern European Restaurant   Dumpling Restaurant             Donut Shop

In [27]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], 
                                  toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine Cluster

## Cluster 1

In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, 
                   toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]][:5]

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
1    The Danforth West, Riverdale      Greek Restaurant           Coffee Shop   
2  The Beaches West, India Bazaar                  Park      Sushi Restaurant   
3                 Studio District                  Café           Coffee Shop   
5                Davisville North                 Hotel                  Park   
6              North Toronto West        Clothing Store           Coffee Shop   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
1    Italian Restaurant            Restaurant        Ice Cream Shop   
2            Board Shop               Brewery    Italian Restaurant   
3             Gastropub                Bakery               Brewery   
5                   Gym        Breakfast Spot        Sandwich Place   
6           Yoga Studio   Sporting Goods Shop    Salon / Barbershop   

    6th Most Common Venue 7th Most Common Venue    8th Most Common Venue  \
1  Furniture / Home Store           Yoga Studio                      Pub   
2            Liquor Store          Burger Joint            Burrito Place   
3      Italian Restaurant   American Restaurant  Comfort Food Restaurant   
5       Food & Drink Shop      Department Store            Women's Store   
6              Restaurant   Rental Car Location                     Café   

  9th Most Common Venue 10th Most Common Venue  
1                Bakery           Dessert Shop  
2        Ice Cream Shop   Fast Food Restaurant  
3        Sandwich Place            Cheese Shop  
5            Donut Shop       Doner Restaurant  
6    Chinese Restaurant     Mexican Restaurant

## Cluster 2

In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, 
                   toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]][:5]

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
8   Moore Park, Summerhill East                  Park            Playground   
10                     Rosedale                  Park            Playground   

   3rd Most Common Venue 4th Most Common Venue        5th Most Common Venue  \
8           Tennis Court            Restaurant             Cuban Restaurant   
10                 Trail          Cupcake Shop  Eastern European Restaurant   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
8             Donut Shop      Doner Restaurant               Dog Run   
10   Dumpling Restaurant            Donut Shop      Doner Restaurant   

   9th Most Common Venue 10th Most Common Venue  
8         Discount Store                  Diner  
10               Dog Run         Discount Store

## Cluster 3

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, 
                   toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]][:5]

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
4  Lawrence Park              Bus Line                  Lake   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
4           Swim School                  Park  Colombian Restaurant   

     6th Most Common Venue 7th Most Common Venue        8th Most Common Venue  \
4  Comfort Food Restaurant     Electronics Store  Eastern European Restaurant   

  9th Most Common Venue 10th Most Common Venue  
4   Dumpling Restaurant             Donut Shop

## Cluster 4

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, 
                   toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]][:5]

Neighborhood    1st Most Common Venue 2nd Most Common Venue  \
22     Roselawn  Health & Beauty Service                  Pool   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
22                Garden         Women's Store          Dance Studio   

          6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
22  Eastern European Restaurant   Dumpling Restaurant            Donut Shop   

   9th Most Common Venue 10th Most Common Venue  
22      Doner Restaurant                Dog Run

## Cluster 5

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, 
                   toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]][:5]

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
0  The Beaches                 Trail     Health Food Store   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0                   Pub         Women's Store          Dance Studio   

         6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0  Eastern European Restaurant   Dumpling Restaurant            Donut Shop   

  9th Most Common Venue 10th Most Common Venue  
0      Doner Restaurant                Dog Run